In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
import datetime as dt
import pandas as pd
from StringIO import StringIO

In [2]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [3]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [4]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

In [5]:
ncs = allURL[0]
dtime2 = netCDF4.num2date(ncs[0:],units = "days since 1985-01-01 00:00:00 UTC", calendar='gregorian')
print dtime2

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: unorderable dtypes; returning scalar but in the future this will be an error
  


AttributeError: 'bool' object has no attribute 'all'

In [6]:
boxLat = []
boxLon = []
boxWh = []
boxT = []

In [7]:
#Cyclone coordinates for marcia
latmin = -19.0
latmax = -18.0  
lonmin = 150.0 
lonmax = 151.0

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')

In [18]:
# Marcia dates
start_date = dt.datetime(1985,1,1)
end_date = dt.datetime(2019,2,21)

#start_date = dt.datetime(2010,1,1)
#end_date = dt.datetime(2010,12,31)


In [19]:
for u in range (len(allURL)):
    urlON = allURL[u]
    for k in range (len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]

        # Get desired time interval  
        time_var = ncs.variables['TIME']
        tt = ncs.variables['TIME'][:]
        
        timing = netCDF4.num2date(tt,time_var.units)
        
        
        for k in range(len(timing)):
            if timing[k] >= start_date and timing[k] <= end_date:
                if lats[k]>latmin and lats[k]<latmax and lons[k]>lonmin and lons[k]<lonmax:
                    boxLat.append(lats[k])
                    boxLon.append(lons[k])
                    boxWh.append(wh[k])
                    boxT.append(tt[k])
                    #print timing[k]
                    
#         id_2 = netCDF4.date2index(end_date,time_var,select='nearest')
#         diff3 = start_date-netCDF4.num2date(tt[id_2],time_var.units)
        
#         diff2 = end_date-netCDF4.num2date(tt[id_2],time_var.units)
        #dh1 = (diff1.seconds + diff1.days * 24 * 3600)/3600.
#         dh2 = (diff2.seconds + diff2.days * 24 * 3600)/3600. #diff in end date (actual vs nearest) in days
#         dh3 = (diff3.seconds + diff3.days * 24 * 3600)/3600. #diff btn start date and nearest end date in days

#         id_st = -1
#         id_et = -1
#         if dh3<0.:
#             id_st = netCDF4.date2index(start_date,time_var,select='after')
            
#         if dh2>-24.:
#             id_et = netCDF4.date2index(end_date,time_var,select='before')
            
#         if id_st<id_et:
#             print id_st,id_et,netCDF4.num2date(tt[id_st],time_var.units)
#             print netCDF4.num2date(tt[id_et],time_var.units)
#             # Check if satellite points are in the box
#             id1 = lats>latmin
#             id2 = lats<latmax
#             id3 = lons>lonmin
#             id4 = lons<lonmax
#             id1 = id1*1
#             id2 = id2*1
#             id3 = id3*1
#             id4 = id4*1
#             tot = id1+id2+id3+id4
            
#             # If this is the case then the sum should be 4
#             inside = np.where(tot==4)[0]
#             idtime = np.arange(id_st,id_et+1)

#             idin = np.intersect1d(inside, idtime)

#             if len(idin)>0:
#                 boxLat.append(lats[idin])
#                 boxLon.append(lons[idin])
#                 boxWh.append(wh[idin])
#                 boxT.append(tt[idin])

In [20]:
len(boxT)

38151

In [21]:
for k in range(len(boxLat)):
    if k == 0:
        lat = boxLat[k]
        lon = boxLon[k]
        wh = boxWh[k]
        tt = boxT[k]
    else:
        lat = np.append(lat,boxLat[k])
        lon = np.append(lon,boxLon[k])
        wh = np.append(wh,boxWh[k])
        tt = np.append(tt,boxT[k])

In [22]:
print len(wh)
wh

38151


masked_array(data=[2.19800019, 2.19800019, 2.22200012, ..., 0.88600004,
                   0.88600004, 0.78000003],
             mask=False,
       fill_value=1e+20)

In [23]:
print 'records for Hydrographers:', len(boxWh)

records for Hydrographers: 38151


In [24]:
#wave height pos values

ids = np.where(wh>0)[0]
whpos = wh[ids]
len(whpos)

38015

In [25]:
print np.mean(whpos)

1.5434551306822604


In [26]:
print 'mean whpos', np.mean(whpos)
print 'max whpos',max(whpos)
print 'min whpos', min (whpos)

#try diff wave names
print 'mean wave height', np.mean(wh)

print 'min lon',min(lon)
print 'max lon',max(lon)

print 'min lat', min(lat)
print 'max lat',max(lat)

print min(netCDF4.num2date(tt,time_var.units))
print min (netCDF4.num2date(tt[ids],time_var.units))
print max (netCDF4.num2date(tt,time_var.units))
print max (netCDF4.num2date(tt[ids],time_var.units))

mean whpos 1.5434551306822604
max whpos 28.41200065612793
min whpos 0.004000000189989805
mean wave height 1.537953049537001
min lon 150.00024
max lon 150.99998
min lat -18.999968
max lat -18.00001
1985-04-09 08:39:51.945317
1985-04-09 08:39:51.945317
2019-02-17 15:25:05.484371
2019-02-17 15:25:05.484371


In [ ]:
for k in range(len(tt)):
    if wh[k]>0:
        print netCDF4.num2date(tt[k],time_var.units)

In [ ]:
tsort = np.sort(tt)
for k in range(len(tsort)):
    print netCDF4.num2date(tsort[k],time_var.units)